In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
# Функция сохранения в файл ответа, представленного массивом
def save_answerArray(task,array):
    with open("C5W4-" + str(task)+".txt","w") as fout:
        fout.write(" ".join([str(el) for el in array]))

In [4]:
df_train = pd.read_table('coursera_sessions_train.txt', sep=';', names=['viewed', 'bought'])
df_test = pd.read_table('coursera_sessions_test.txt', sep=';', names=['viewed', 'bought'])
df_test.head()
#df_train.info()

,viewed,bought
0,"6,7,8",NaN
1,"13,14,15",NaN
2,"22,23",NaN
3,"28,29,30,31,32,33",NaN
4,"40,41",NaN


In [4]:
viewed_train =[]
for i in df_train.viewed:
    viewed_train.extend(i.split(','))
df_train.dropna(subset=['bought'], inplace=True)
df_test.dropna(subset=['bought'], inplace=True)    
bought_train =[]
for i in df_train.bought:
    bought_train.extend(i.split(','))

In [5]:
c_viewed = collections.Counter()
c_bought = collections.Counter()
for i in viewed_train:
    c_viewed[i] += 1
for i in bought_train:
    c_bought[i] += 1




In [90]:
l = '2,60,8,59,60,7,9'
l1 = [1,1,2,10]
#(list(collections.OrderedDict.fromkeys(l)))
print([x[0]for x in sorted(zip(list(collections.OrderedDict.fromkeys(l)), l1), key=lambda x: (x[1]), reverse=True)])


['0', '6', '2', ',']


In [105]:
def recom(lst, weight):
    lst_n = list(collections.OrderedDict.fromkeys(lst.split(',')))
    r = [x[0] for x in sorted(zip(lst_n,[weight[str(x)]for x in lst_n]), key=lambda x: (x[1]), reverse=True)]
    return ','.join(str(x) for x in (r[0 : min(len(lst_n), 5)]))
def recom_viewed(lst):
    return recom(lst, c_viewed)
def recom_bought(lst):
    return recom(lst, c_bought)

In [102]:
print(recom_viewed(l))
print(c_viewed['8'])


7,8,2,9,60
225


In [108]:
df_train['re_viewed'] = df_train['viewed'].apply(recom_viewed)
df_train['re_bought'] = df_train['viewed'].apply(recom_bought)
df_test['re_viewed'] = df_test['viewed'].apply(recom_viewed)
df_test['re_bought'] = df_test['viewed'].apply(recom_bought)

In [109]:
df_test.head()

,viewed,bought,re_viewed,re_bought
7,"63,68,69,70,66,61,59,61,66,68","66,63","63,68,66,61,59","63,68,69,70,66"
14,"158,159,160,159,161,162",162,"158,162,160,159,161","158,162,160,159,161"
19,"200,201,202,203,204","201,205","204,202,203,200,201","204,202,200,201,203"
34,"371,372,371","371,373","371,372","371,372"
40,422,422,422,422


In [155]:
def prec(b, r, k):
    b_l = b.split(',')
    r_l = r.split(',')
    rez = 0
    for i in range(min(k, len(r_l))):
        if r_l[i] in b_l:
            rez += 1
    return rez / k
def prec_1(b, r):
    return prec(b, r, 1)
def prec_5(b, r):
    return prec(b, r, 5)

def rec(b, r, k):
    b_l = b.split(',')
    r_l = r.split(',')
    rez = 0
    for i in range(min(k, len(r_l))):
        if r_l[i] in b_l:
            rez += 1
    return rez / len(b_l)        

In [130]:
b = '1,2,3'
r = '3,5'
print(prec_1(b, r))

1.0


In [156]:

for i in df_train.index:
    df_train.loc[[i],['prec_1_v']] = prec_1(df_train['bought'][i], df_train['re_viewed'][i])
    df_train.loc[[i],['prec_5_v']] = prec_5(df_train['bought'][i], df_train['re_viewed'][i])
    df_train.loc[[i],['rec_1_v']] = rec(df_train['bought'][i], df_train['re_viewed'][i], 1)
    df_train.loc[[i],['rec_5_v']] = rec(df_train['bought'][i], df_train['re_viewed'][i], 5)

In [149]:
df_train.head()

,viewed,bought,re_viewed,re_bought,prec_1_v,prec_5_v,rec_1_v,rec_5_v
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63","63,64,60,61,65","60,63,67,59,61",1.0,0.40,0.333333,0.666667
10,"84,85,86,87,88,89,84,90,91,92,93,86",86,"85,93,89,90,84","86,85,93,84,87",0.0,0.00,0.000000,0.000000
19,"138,198,199,127",199,"127,138,198,199","138,199,127,198",0.0,0.25,0.000000,1.000000
30,"303,304,305,306,307,308,309,310,311,312",303,"303,306,304,307,309","303,304,305,306,307",1.0,0.20,1.000000,1.000000
33,"352,353,352",352,"352,353","352,353",1.0,0.50,1.000000,1.000000


In [163]:
#save_answerArray(1,[round(df_train.rec_1_v.mean(), 2), round(df_train.prec_1_v.mean(), 2),
#                    round(df_train.rec_5_v.mean(), 2), round(df_train.prec_5_v.mean(), 2)])
print(round(df_train.rec_1_v.mean(), 2), round(df_train.prec_1_v.mean(), 2),
                    round(df_train.rec_5_v.mean(), 2), round(df_train.prec_5_v.mean(), 2))    

0.44 0.51 0.82 0.21


In [158]:
for i in df_test.index:
    df_test.loc[[i],['prec_1_v']] = prec_1(df_test['bought'][i], df_test['re_viewed'][i])
    df_test.loc[[i],['prec_5_v']] = prec_5(df_test['bought'][i], df_test['re_viewed'][i])
    df_test.loc[[i],['rec_1_v']] = rec(df_test['bought'][i], df_test['re_viewed'][i], 1)
    df_test.loc[[i],['rec_5_v']] = rec(df_test['bought'][i], df_test['re_viewed'][i], 5)

In [169]:
#save_answerArray(2,[round(df_test.rec_1_v.mean(), 2), round(df_test.prec_1_v.mean(), 2),
#                    round(df_test.rec_5_v.mean(), 2), round(df_test.prec_5_v.mean(), 2)])
print(round(df_test.rec_1_v.mean(), 2), round(df_test.prec_1_v.mean(), 2),
                    round(df_test.rec_5_v.mean(), 2), round(df_test.prec_5_v.mean(), 2))

0.42 0.48 0.8 0.2


In [160]:
for i in df_train.index:
    df_train.loc[[i],['prec_1_b']] = prec_1(df_train['bought'][i], df_train['re_bought'][i])
    df_train.loc[[i],['prec_5_b']] = prec_5(df_train['bought'][i], df_train['re_bought'][i])
    df_train.loc[[i],['rec_1_b']] = rec(df_train['bought'][i], df_train['re_bought'][i], 1)
    df_train.loc[[i],['rec_5_b']] = rec(df_train['bought'][i], df_train['re_bought'][i], 5)

In [165]:
#save_answerArray(3,[round(df_train.rec_1_b.mean(), 2), round(df_train.prec_1_b.mean(), 2),
#                    round(df_train.rec_5_b.mean(), 2), round(df_train.prec_5_b.mean(), 2)])
print(round(df_train.rec_1_b.mean(), 2), round(df_train.prec_1_b.mean(), 2),
                    round(df_train.rec_5_b.mean(), 2), round(df_train.prec_5_b.mean(), 2))

0.69 0.8 0.93 0.25


In [166]:
for i in df_test.index:
    df_test.loc[[i],['prec_1_b']] = prec_1(df_test['bought'][i], df_test['re_bought'][i])
    df_test.loc[[i],['prec_5_b']] = prec_5(df_test['bought'][i], df_test['re_bought'][i])
    df_test.loc[[i],['rec_1_b']] = rec(df_test['bought'][i], df_test['re_bought'][i], 1)
    df_test.loc[[i],['rec_5_b']] = rec(df_test['bought'][i], df_test['re_bought'][i], 5)

In [168]:
#save_answerArray(4,[round(df_test.rec_1_b.mean(), 2), round(df_test.prec_1_b.mean(), 2),
#                    round(df_test.rec_5_b.mean(), 2), round(df_test.prec_5_b.mean(), 2)])
print(round(df_test.rec_1_v.mean(), 2), round(df_test.prec_1_v.mean(), 2),
                    round(df_test.rec_5_b.mean(), 2), round(df_test.prec_5_b.mean(), 2))

0.42 0.48 0.82 0.21
